FASE 1: Realización de un Script para la limpieza y organización del Data Set.

In [37]:
import pandas as pd # Estas son las librerías a utilizar para el análisis, únicamente pandas, numpy es una dependencia de pandas.
import numpy as np

In [38]:
df = pd.read_csv('Prueba analisis de Datos - Rettain - Data.csv') # De esta forma se cargan los archivos .csv, pero también puede ser archivo .excel, .sql, etc.

df # Ya que cargamos la variable, podemos mostrarla tras "llamarla"

# Limpiando 'Campaign Name'

df['Campaign Name'] = df['Campaign Name'].str.strip() # Aquí se eliminan los espacios en blanco al inicio y al final de los nombres de campaña.

# Limpiando y formateando la columna 'Send Time'

df = df.drop(columns=['Send Time']) # Aquí se elimina la columna "Send Time" del DataFrame porque es una copia inutilizada.
df = df.rename(columns={'Send Time.1':'Send Time'}) # Aquí se renombra la columna "Send Time.1" a "Send Time" para facilitar su uso posterior.
df['Send Time'] = pd.to_datetime(df['Send Time'],utc=True).dt.tz_localize(None).dt.floor('s') # Aquí se convierte la columna "Send Time" al formato de fecha y hora adecuado, evitando los milisegundos y las zonas horarias.
df['Time_Interval'] = df['Send Time'].dt.hour.apply(lambda x: f"{x:02d}:00–{x+1:02d}:00") # Separar en intérvalos de 1 hora como lo dicho en el archivo.
df['Day_of_Week'] = df['Send Time'].dt.day_name() # Aquí se crea una nueva columna "day_of_week" que contiene el nombre del día de la semana correspondiente a cada fecha en "Send Time".

# Limpiando las columnas numéricas y convirtiéndolas a tipo numérico: 

numeric_columns = ['Recipients', 'Revenue', 'First Purchase', 'Returning Customer'] # Aquí se define una lista con los nombres de las columnas numéricas a limpiar.

for column in numeric_columns:
    df[column] = df[column].astype(str).str.replace(',', '').str.replace('$', '') # Aquí se eliminan los caracteres no numéricos como comas y signos de dólar.
    df[column] = pd.to_numeric(df[column], errors='coerce') # Aquí se convierte la columna a tipo numérico, manejando errores si los hay (signos - ).
    df[column] = df[column].fillna(0) # Aquí se rellenan los valores NaN (NULLS o Nulos) con 0 para evitar problemas en análisis posteriores.


# Limpiando y formateando las columnas "porcentuales":

percentage_columns = ['Open Rate', 'Click Rate', 'CTR', 'CVR'] # Aquí se define una lista con los nombres de las columnas porcentuales a limpiar.

for column in percentage_columns:
    df[column] = df[column].str.replace('%', '') # Aquí se eliminan los signos de porcentaje.
    df[column] = pd.to_numeric(df[column], errors='coerce') # Aquí se convierte la columna a tipo numérico, manejando errores si los hay. 
    df[column] = df[column] / 100 # Aquí se convierte el valor a su forma decimal dividiendo entre 100, para que sea válido al momento del análisis.
    df[column] = df[column].fillna(0) # Aquí se rellenan los valores NaN (NULLS o Nulos) con 0 para evitar problemas en análisis posteriores.

df.to_csv('Prueba Analisis de Datos - Rettain - Limpio.csv', index=False) # Aquí se exporta el DataFrame limpio a un nuevo archivo .csv sin incluir los índices.

FASE 2: Realización de sub-tablas para la resolución de preguntas.

In [39]:
# P1. Revenue per Recipient por día de la semana (Lunes, Martes, Miércoles, etc)

p1 = df.groupby('Day_of_Week')[['Revenue', 'Recipients']].sum()
# Esta primera tabla consistirá únicamente de las columnas Revenue y Recipients, que son agrupadas por los días de la semana.

p1['RPR'] = p1['Revenue'] / p1['Recipients'] # Ahora realizamos la métrica RPR, obtenida tras dividir la columna Revenue por Recipients.
p = p1.fillna(0) # Valores nulos reemplazados por 0.

p1.to_csv('p1.csv')

p1

,Revenue,Recipients,RPR
Day_of_Week,,,
Friday,210387.0,2571949,0.081801
Monday,134559.0,1766624,0.076167
Saturday,144257.0,1593288,0.090540
Sunday,129603.0,1882345,0.068852
Thursday,75352.0,1468925,0.051297
Tuesday,159053.0,2409247,0.066018
Wednesday,101489.0,1517180,0.066893


In [40]:
# P2. Revenue por hora de envío (8-9; 9-10; 10-11, etc)

p2 = df.groupby('Time_Interval')['Revenue'].sum().sort_values(ascending=False)
# Esta segunda tabla es más sencilla, únicamente necesitando las horas y el revenue.

p2.to_csv('p2.csv')

p2

Time_Interval
12:00–13:00    241696.0
09:00–10:00    121159.0
18:00–19:00    104286.0
10:00–11:00     89970.0
15:00–16:00     65642.0
11:00–12:00     65566.0
14:00–15:00     64878.0
17:00–18:00     50459.0
06:00–07:00     45253.0
16:00–17:00     43722.0
08:00–09:00     23237.0
13:00–14:00     13687.0
19:00–20:00     13680.0
07:00–08:00      5665.0
20:00–21:00      2812.0
22:00–23:00      2676.0
21:00–22:00       312.0
Name: Revenue, dtype: float64

In [41]:
# P3. Mejor hora para mandar correos por día de la semana (basado en Revenue per Recipient). (Ej: Lunes 8-9am; Martes 3-4pm; Miercoles 10-11am)

p3_revenue = df.pivot_table(index='Time_Interval', columns='Day_of_Week', values='Revenue', aggfunc='sum').replace([np.inf, -np.inf], np.nan).fillna(0)
p3_recipients = df.pivot_table(index='Time_Interval', columns='Day_of_Week', values='Recipients', aggfunc='sum').replace([np.inf, -np.inf], np.nan).fillna(0)
p3_rpr = (p3_revenue / p3_recipients).replace([np.inf, -np.inf], np.nan).fillna(0)
p3_rpr = p3_rpr # En esta tabla hay valores infinitos y nulos, los reemplacé todos por 0.

# Estas tres son métricas distintas, pero cada una se utilizará para la misma pregunta, lo que queríamos saber es el mejor día y hora, basado en un RPR.

# En esta se requirió hacer una tabla dinámica; donde la hora se encargaría de "agrupar" la columna, 
# y la principal columna sería los días de la semana, habiendo otra columna que contendría la suma total de revenue o recipients.

p3_revenue.to_csv('p3_revenue.csv')
p3_recipients.to_csv('p3_recipients.csv')
p3_rpr.to_csv('p3_rpr.csv')

p3_rpr

Day_of_Week,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday
Time_Interval,,,,,,,
06:00–07:00,0.180580,0.000000,0.000000,0.000000,0.000000,0.040805,0.000000
07:00–08:00,0.272415,0.000000,0.000000,0.046736,0.000000,0.000000,0.000000
08:00–09:00,0.000000,0.000000,0.078754,0.017981,0.060210,0.000000,0.106703
09:00–10:00,0.138357,0.000000,0.023705,0.047005,0.045908,0.062750,0.038220
10:00–11:00,0.111706,0.038856,0.058099,0.081758,0.000000,0.023070,0.098705
11:00–12:00,0.075969,0.014628,0.099736,0.035598,0.040157,0.198055,0.032325
12:00–13:00,0.036429,0.121557,0.216439,0.075931,0.069346,0.026564,0.155329
13:00–14:00,0.000000,0.000000,0.000000,0.017029,0.053539,0.000000,0.026859
14:00–15:00,0.060635,0.000000,0.000000,0.036645,0.000000,0.141269,0.047943


In [42]:
# P456. Mejor hora para tener el mayor click rate, open rate y CVR (sin importar el día de la semana).

p456 = df.groupby('Time_Interval')[['Click Rate', 'Open Rate', 'CVR']].mean()
# Esta tabla solo tiene el valor promedio de las columnas [['Click Rate', 'Open Rate', 'CVR']],
# Agrupado por las horas.
p456 = p456.fillna(0)

p456.to_csv('p456.csv')

p456


,Click Rate,Open Rate,CVR
Time_Interval,,,
06:00–07:00,0.011067,0.712467,0.001233
07:00–08:00,0.005950,0.357650,0.000600
08:00–09:00,0.006918,0.483473,0.000436
09:00–10:00,0.009105,0.572910,0.000505
10:00–11:00,0.011763,0.557432,0.001095
11:00–12:00,0.010743,0.573679,0.000614
12:00–13:00,0.018500,0.632155,0.001813
13:00–14:00,0.007475,0.645350,0.000350
14:00–15:00,0.012250,0.608800,0.000740


In [43]:
# P78. Mejor día de la semana para maximizar First Purchase Revenue per Recipient y Returning Customer Revenue per Recipient

p7 = df.groupby('Day_of_Week')[['First Purchase', 'Recipients']].sum()
p7['FP RPR'] = p7['First Purchase'] / p7['Recipients']

p8 = df.groupby('Day_of_Week')[['Returning Customer', 'Recipients']].sum()
p8['RC RPR'] = p8['Returning Customer'] / p8['Recipients']

# Ambas columnas son similares a la primera, solo que esta vez en lugar de utilizar al revenue general, se utilizan dos columnas distintas;
# [['Returning Customer', 'First Purchase']] pero es en teoría la misma fórmulación.

p7.to_csv('p7.csv')
p8.to_csv('p8.csv')

print(f'{p7}\n{p8}')

             First Purchase  Recipients    FP RPR
Day_of_Week                                      
Friday              69172.0     2571949  0.026895
Monday              39153.0     1766624  0.022163
Saturday            39653.0     1593288  0.024888
Sunday              45464.0     1882345  0.024153
Thursday            25286.0     1468925  0.017214
Tuesday             54596.0     2409247  0.022661
Wednesday           39002.0     1517180  0.025707
             Returning Customer  Recipients    RC RPR
Day_of_Week                                          
Friday                 141205.0     2571949  0.054902
Monday                  95405.0     1766624  0.054004
Saturday               104605.0     1593288  0.065654
Sunday                  84138.0     1882345  0.044699
Thursday                50067.0     1468925  0.034084
Tuesday                104457.0     2409247  0.043357
Wednesday               62490.0     1517180  0.041188
